# 1. 엑셀데이터 불러오기

In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import numpy as np
import re

df_MCN = pd.read_csv('phone_info.csv')


In [5]:
df_MCN.iloc[1]

pl_id                       2
pl_maker                apple
pl_group                apple
pl_model_code           A2215
pl_name             아이폰11 Pro
pl_model_name    iphone11 Pro
Name: 1, dtype: object

# 2. 일단 한페이지에서 파싱하기

In [4]:
import time

def search_by_category(i,option):
    if option == "skt":
        url1 = 'https://search.shopping.naver.com/search/category?catId=50000246&frm=NVSHMDL&iq='
    elif option == "kt":
        url1 = 'https://search.shopping.naver.com/search/category?catId=50000247&frm=NVSHMDL&iq='
    elif option == "lg":
        url1 = 'https://search.shopping.naver.com/search/category?catId=50000248&frm=NVSHMDL&iq='
    url2 = '&origQuery&pagingIndex=1&pagingSize=40&productSet=model&query&sort=rel&timestamp=&viewType=list'
    driver.get(url1+df_MCN.iloc[i][4]+url2)
    driver.implicitly_wait(3)
    
    #페이지에 있는 링크 다 긁어오기, 일단 스크롤기능 추가안해서 5개만 가져옴
    req = driver.page_source
    soup=BeautifulSoup(req, 'html.parser')
    div_list = soup.select(".basicList_info_area__17Xyo")
    
    link_list = []
    for i in range(len(div_list)):
        if div_list[i].select("div.basicList_info_area__17Xyo > div.basicList_etc_box__1Jzg6 > a") == []:
#             print("NONE!!")
            pass
        else:
#             print("good")
            link_list.append(div_list[i].select(".basicList_link__1MaTN")[0])
    

    for link in link_list:
        url = 'window.open("' + link.get('href') +'");'
        driver.execute_script(url)
        driver.switch_to.window(driver.window_handles[-1])
        req = driver.page_source
        soup=BeautifulSoup(req, 'html.parser')
        review_list = soup.select(".reviewItems_list_review__1sgcJ")
        review_list = review_list[0].find_all("li")
        
        for review in review_list:
            area1 = review.select(".reviewItems_etc_area__2P8i3")
            area1 = area1[0].find_all("span")
            
            star = area1[0].text.replace("평점","") #별점
            market = area1[3].text #구입한 오픈마켓 이름
            masking_id = area1[4].text #아이디이름(부분가림)
            ymd = area1[5].text #구입년월일
            
            area2 = review.select(".reviewItems_review_text__2Bwpa")
            title = area2[0].select(".reviewItems_title__39Z8H")[0].text
            text = area2[0].select(".reviewItems_text__XIsTc")[0].text
            
            print(star,market,masking_id,ymd,title,text)
        
        # 현재 탭 닫기
        driver.close()
        # 맨 처음 탭으로 변경(0번 탭)
        driver.switch_to.window(driver.window_handles[0])

# 초기화 --------------------------------------------
chrome_options = Options()
chrome_options.add_argument("--headless")
driver  = webdriver.Chrome('chromedriver')
driver.implicitly_wait(3)


# for i in range(len(df_MCN)):
for i in range(12,14):
    print("=================",i,"skt","=================")
    search_by_category(i,"skt")
    print("=================",i,"kt","=================")
    search_by_category(i,"kt")
    print("=================",i,"lg","=================")
    search_by_category(i,"lg")
            
        
        
        


================= 12 skt =================
5 SSG닷컴 sm****** 18.04.12. 기기변경으로 아이폰구매하고싶? 기기변경으로 아이폰구매하고싶었는데제가알아본곳중에서 조건도 제일 괜찮고 저렴해서 구매했어요현금완납이라서 너무 잘산것같네요그전에는 아이폰6사용했는데 6s로 그래도 한단계 업그레이드 했어요 ^^휴대폰 너무 좋구요. 기기변경으로 유심그대로 사용하는걸로했는데 휴대폰도 안끊기고 그다음날 오후에 바로받아서 새폰 사용중이예요 128기가라서 용량걱정도없을것같네요^^!!재조년월도 관심있게생각안했는데 18년1월이라고 되어져있어요.!!! 넘나맘이듭니다 색상은 현재 스페이스그레이 이색상만 된다고하셨구요~~~! 원래사용하던 요금제라서 요금이 더많이나가고 그런것도 없어요~~~^^기기변경으로 아이폰 저렴하게 구매하고싶으신분들께 강추?? 2년동안 6s 이쁘게써보겠습니다~~~~~!!!^^상담하시던분도 너무 친절하게 잘 응대해주셨구요가입신청서 작성하는것도 어렵지않아요 ! 혹시 걱정이신분들은 잘알려주시니까 그대로 따라하시면됩니당^^ 넘넘좋아요 ㅠㅠ주변분들한테도 언능 전파해야겠습니다 !!!!!!!!!!!!!!
4 인터파크 topz**** 18.07.29. 아이폰 6s 구매 인터넷으로 핸드폰구매를 처음해서 약간불안했지만선택한 요금제로 카톡으로 개통했지만 새폰임에도 음량에문제가 있어 아이폰 AS센터와 상담후 이런저런 기능을 전부 테스트 중입니다 기본 10자 이상 작성시 100P 즉시지급 이미지 등록시 100P 즉시 추가지급개당 상품 가격이 5,000원 이하 상품은 적립대상에서 제외되며,리뷰 포인트는 최초 1회 작성시에만 지급됩니다.같은 단어가 반복된 내용은 임의로 삭제되며, 지급된 포인트가 회수될 수 있습니다. 다른 고객님들에게 유용한 상품 리뷰를 작성해주세요.
5 인터파크 koo***** 18.07.30. 잘 받았어요 웹상에서 신청서작성하기가 좀 어려웠네요. 좋은 상품 저렴히 구입했습니다. 감사요. 기본 10자 이상 작성시 100P 즉시지급 이미

5 롯데홈쇼핑 mer***** 16.07.22. 좋습니다좋습니다 좋아요 굿 좋습니다좋습니다 좋아요 굿
5 11번가 sw****** 15.12.01. 너무마음에듭니다.적극추천합니다. 아이폰이 처음인데 적응이 힘드네요.직접수령했는데 재고 확보해줘서 감사했습니다.선택약정할인 전환안되게 125일 락걸어서 좀 아쉬웠습니다.많이파세요
5 11번가 mi****** 15.11.10. 싸게 잘 구입했습니다. 당일 해피콜도 빠르게 진행되어 택배로 잘 도착했습니다.기기에도 손상없고 문제없이 잘 도착했구요~ 역시 6s+ 빠릿빠릿하고 너무 마음에 드네요~기기변경으로 구입하시는분들 적극추천합니다^^ 
5 인터파크 gule**** 17.05.25. 저렴하게 잘 구매했어요 어른들 휴대폰 바꿔드리는데, 큰 액정이 좋으시다고 하셔서 아이폰 플러스로 했네요인터넷으로 휴대폰 사는거 어려울 거라고 걱정했는데, 알아서 잘 해주시네요.엄마 아빠 친구아빠이렇게 3대나 구매했네요!!!!! 
5 인터파크 gule**** 17.05.25. 저렴하게 잘샀습니다 인터넷으로 휴대폰 사는거 복잡할까봐 걱정했는데, 알아서 잘 해주시네요3대나 구매했네요.많이 파세요. 
4 G마켓 dosh**** 16.03.27. 핸드폰사기어렵군요 아.. 핸드폰사는건 너무 어려운 일이군요... ㅠㅠㅠㅠㅠㅠ
================= 12 kt =================
================= 12 lg =================
5 인터파크 salt**** 17.10.16. 폰이 갈때가 되어서 신랑이 바꿔줬는데... 좋으네요 알아서 신랑이 동기화까지 다 시켜주고 으흐 기존 64G도 용량이 풀이라 128G로 업그레이드에 직접 내방까지해서 주변분들도 잘 샀다고 했다고 사은품도 잘 챙겨주시고헌데 젤 중요한 개통이 안되어서;;;; 신랑이 대리점 두곳을 돌아다닌 결과 뜯어보고 유심칩이 불량이라고해서 새로 바꿔주어서 들고왔는데 고생시킨듯해서 미안하더군요그래서 별을 몇개 뺐네요~
5 인터파크 apri**** 17.07.14

In [22]:
df_MCN.tail(30)

,pl_id,pl_maker,pl_group,pl_model_code,pl_name,pl_model_name
209,220,samsung,note_series,SM-N986,갤럭시노트20 울트라,GalaxyNote20 Ultra
210,221,lg,q_series,LM-Q310,Q31,LG Q31
211,222,lg,q_series,LM-Q520,Q52,LG Q52
212,223,lg,q_series,LM-F100,윙,LG Wing
213,224,samsung,a_series,SM-A102,갤럭시 A10e,Galaxy A10e
214,225,lg,lg,LG-D821,넥서스5,Google Nexus5 16G
215,226,lg,lg,LG-F100,옵티머스 뷰,Optimus VU
216,227,lg,g_series,LG-F180,옵티머스 G,Optimus G
217,228,lg,lg,LG-F200,옵티머스 뷰2,Optimus VU2
218,229,lg,g_series,LG-F220,옵티머스 GK,LG Optimus GK


In [57]:
df_MCN.iloc[1][4]

'아이폰11 Pro'

# 불러온 엑셀데이터로 차례대로 파싱

In [35]:
 my_dict = {
    "maker": "",
    "group": "",
    "code": "",
    
    "name": "",
    "model_name": "",
    "storage": "",
    "color_name": "",
    
    "hidden":"",
    "storage_hidden": "",
    "color_hidden": "", #색상사용여부
    "class_hidden": "", #등급사용여부?
    
    "size": "",
    "display_inch": "",
    "display_cm": "",
    "weight": "",
    "battery": "",
    "front_camera": "",
    "back_camera": "",
    
    "price": "",
    "score": "", #별점
    "summary": "" #스펙요약
    }
crawl_data = pd.DataFrame(my_dict,index=[0])

In [36]:
def hasxpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
        return True
    except:
        return False

In [39]:
# 초기화 --------------------------------------------
chrome_options = Options()
chrome_options.add_argument("--headless")

driver  = webdriver.Chrome('./chromedriver_win32/chromedriver')
driver.implicitly_wait(3)

for i in range(len(df_MCN)):
# for i in range(3):
    url1 = "https://review.cetizen.com/review.php?q=phone&just_one=&just_one_name=&just_one_pcat=&keyword_p="
    url2 = "#&p_data=3&p_split=&recnum=10"
    driver.get(url1+df_MCN.iloc[i][2]+url2)
    
    if hasxpath('//*[@id="product_list"]/div/div[3]/div[1]/div[1]/div/a/span') == True:
        driver.find_element_by_xpath('//*[@id="product_list"]/div/div[3]/div[1]/div[1]/div/a/span').click()
        driver.implicitly_wait(10)
        
        name = driver.find_element_by_xpath('/html/body/div[10]/div[3]/div[1]/div[2]/div[1]/div[1]/div[1]/a/span').text
        model_name = driver.find_element_by_xpath('/html/body/div[10]/div[3]/div[1]/div[2]/div[1]/div[2]/span').text
        storage = driver.find_element_by_xpath('//*[@id="product_specview"]/div[1]/form/div[14]/div[2]/span').text
        color_name = ""
        size = driver.find_element_by_xpath('//*[@id="product_specview"]/div[1]/form/div[4]/div[4]/span').text
        display_inch = driver.find_element_by_xpath('//*[@id="product_specview"]/div[1]/form/div[7]/div[2]/span').text
        weight = driver.find_element_by_xpath('//*[@id="product_specview"]/div[1]/form/div[5]/div[2]/span').text
        battery = driver.find_element_by_xpath('//*[@id="product_specview"]/div[1]/form/div[22]/div[2]/span').text
        front_camera = driver.find_element_by_xpath('//*[@id="product_specview"]/div[1]/form/div[19]/div[2]/span').text
        back_camera = driver.find_element_by_xpath('//*[@id="product_specview"]/div[1]/form/div[17]/div[4]/span').text
        price =  driver.find_element_by_xpath('//*[@id="product_specview"]/div[1]/form/div[2]/div[2]/span').text

        my_dict = {
        "maker":df_MCN.iloc[i][0],
        "group": df_MCN.iloc[i][1],
        "code": df_MCN.iloc[i][2],

        "name": name,  #모델한국명
        "model_name": model_name, #모델영문명
        "storage": storage, #용량
        "color_name": "", #모델색상

        "hidden": "", #모델사용여부 => 일단비워두자
        "storage_hidden": "", #용량사용여부 => ''
        "color_hidden": "", #색상사용여부
        "class_hidden": "", #등급사용여부?

        "size": size,
        "display_inch": display_inch,
        "display_cm": str(round(float(display_inch)*2.5399,1)),
        "weight": weight,
        "battery": battery,
        "front_camera": front_camera,
        "back_camera": back_camera,

        "price": price, #출고가
        "score": "", #별점
        "summary": "" #스펙요약
        }
    else:
#         driver.get(url1+df_MCN.iloc[i][1]+url2)
#         if hasxpath('//*[@id="product_list"]/div/div[3]/div[1]/div[1]/div/a/span') == True:
#             driver.find_element_by_xpath('//*[@id="product_list"]/div/div[3]/div[1]/div[1]/div/a/span').click()
#             driver.implicitly_wait(10)
#             size = driver.find_element_by_xpath('//*[@id="product_specview"]/div[1]/form/div[4]/div[4]/span')
#             display_inch = driver.find_element_by_xpath('//*[@id="product_specview"]/div[1]/form/div[7]/div[2]/span')
#             # display_cm = driver.find_element_by_xpath('')
#             weight = driver.find_element_by_xpath('//*[@id="product_specview"]/div[1]/form/div[5]/div[2]/span')
#             battery = driver.find_element_by_xpath('//*[@id="product_specview"]/div[1]/form/div[22]/div[2]/span')
#             front_camera = driver.find_element_by_xpath('//*[@id="product_specview"]/div[1]/form/div[19]/div[2]/span')
#             back_camera = driver.find_element_by_xpath('//*[@id="product_specview"]/div[1]/form/div[17]/div[4]/span')

#             my_dict = {
#             "maker":df_MCN.iloc[i][0],
#             "group": df_MCN.iloc[i][1],
#             "code": df_MCN.iloc[i][2],
#             "size": size.text, 
#             "display_inch": display_inch.text, 
#             "weight": weight.text,
#             "battery": battery.text,
#             "front_camera": front_camera.text,
#             "back_camera": back_camera.text
#             }
#         else:
#              my_dict = {
#             "maker":df_MCN.iloc[i][0],
#             "group": df_MCN.iloc[i][1],
#             "code": df_MCN.iloc[i][2],
#             "size": "", 
#             "display_inch": "", 
#             "weight": "",
#             "battery": "",
#             "front_camera": "",
#             "back_camera": ""
#             }
        my_dict = {
        "maker":df_MCN.iloc[i][0],
        "group": df_MCN.iloc[i][1],
        "code": df_MCN.iloc[i][2],
        "name": "",
        "model_name": "",
        "storage": "",
        "color_name": "",

        "hidden":"",
        "storage_hidden": "",
        "color_hidden": "", #색상사용여부
        "class_hidden": "", #등급사용여부?

        "size": "",
        "display_inch": "",
        "display_cm": "",
        "weight": "",
        "battery": "",
        "front_camera": "",
        "back_camera": "",

        "price": "",
        "score": "", #별점
        "summary": "" #스펙요약
        }
    a= pd.DataFrame(my_dict,index=[0])
    crawl_data = crawl_data.append(a)
    crawl_data = crawl_data.reset_index(drop=True)
    

AttributeError: 'NoneType' object has no attribute 'text'

In [40]:
# display(crawl_data)
# display(a)
# crawl_data = crawl_data.append(a)
# crawl_data = crawl_data.reset_index(drop=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(crawl_data)


,maker,group,code,name,model_name,storage,color_name,hidden,storage_hidden,color_hidden,class_hidden,size,display_inch,display_cm,weight,battery,front_camera,back_camera,price,score,summary
0,,,,,,,,,,,,,,,,,,,,,
1,apple,iphone11,A2221,아이폰11,Apple iPhone 11,64 GB / 128 GB / 256 GB,,,,,,75.7 x 150.9 x 8.3,6.1,15.5,194g,3110 mAh,4032 x 3024 (12.2 메가픽셀),4032 x 3024 (12.2 메가픽셀),"SKT : ￦859,100 | KT : ￦990,000 | LG U+ : ￦990,000",,
2,apple,iphone11 Pro,A2215,아이폰11프로,Apple iPhone 11 Pro,64 GB / 256 GB / 512 GB,,,,,,71.4 x 144 x 8.1,5.8,14.7,188g,3190 mAh,4032 x 3024 (12.2 메가픽셀),4032 x 3024 (12.2 메가픽셀),"SKT : ￦1,174,800 | KT : ￦1,375,000 | LG U+ : ￦...",,
3,apple,iphone11 Pro Max,A2218,아이폰11프로맥스,Apple iPhone 11 Pro Max,64 GB / 256 GB / 512 GB,,,,,,77.8 x 158 x 8.1,6.5,16.5,226g,3500 mAh,4032 x 3024 (12.2 메가픽셀),4032 x 3024 (12.2 메가픽셀),"SKT : ￦1,328,800 | KT : ￦1,529,000 | LG U+ : ￦...",,
4,apple,iphoneXs,A2097,아이폰Xs,Apple iPhone Xs,64 GB / 256 GB / 512 GB,,,,,,70.9 x 143.6 x 7.7,5.8,14.7,177g,2658 mAh,3088 x 2316 (7.2 메가픽셀),4032 x 3024 (12.2 메가픽셀),"KT : ￦891,000 | SKT : ￦891,000 | LG U+ : ￦1,36...",,
5,apple,iphoneXs Max,A2101,아이폰Xs 맥스,Apple iPhone Xs Max,64 GB / 256 GB / 512 GB,,,,,,77.4 x 157.5 x 7.9,6.5,16.5,208g,3174 mAh,3088 x 2316 (7.2 메가픽셀),4032 x 3024 (12.2 메가픽셀),"SKT : ￦940,500 | KT : ￦1,496,000 | LG U+ : ￦1,...",,
6,apple,iphoneXr,A2105,아이폰XR,Apple iPhone XR,64 GB / 128 GB / 256 GB,,,,,,75.7 x 150.9 x 8.3,6.1,15.5,194g,2942 mAh,3088 x 2316 (7.2 메가픽셀),4032 x 3024 (12.2 메가픽셀),"KT : ￦841,500 | LG U+ : ￦990,000 | SKT : ￦841,500",,
7,apple,iphoneX,A1901,아이폰X,Apple iPhone X,64 GB / 256 GB,,,,,,70.9 x 143.6 x 7.7,5.8,14.7,174g,2716 mAh,3088 x 2320 (7.2 메가픽셀),4032 x 3024 (12.2 메가픽셀),"LG U+ : ￦1,155,000 | KT : ￦847,000 | SKT : ￦1,...",,
8,apple,iphone8,A1905,아이폰8,Apple iPhone 8,64 GB / 256 GB,,,,,,67.3 x 138.4 x 7.3,4.7,11.9,148g,1821 mAh,(0 메가픽셀),4032 x 3024 (12.2 메가픽셀),"LG U+ : ￦946,000 | KT : ￦799,700 | SKT : ￦753,500",,
9,apple,iphone8 Plus,A1897,아이폰8플러스,Apple iPhone 8 Plus,64 GB / 256 GB,,,,,,78.1 x 158.4 x 7.5,5.5,14.0,202g,2691 mAh,(0 메가픽셀),4032 x 3024 (12.2 메가픽셀),"LG U+ : ￦1,076,900 | KT : ￦847,000 | SKT : ￦1,...",,


In [90]:
crawl_data.to_csv('crawl_data.csv', # file path, file name
                  sep=',')   # seperator, delimiter (구분자)
#     ...: na_rep='NaN')   # missing data representation (결측값 표기)



In [ ]:
# 모델명이 같은 데이터들 스펙을 모두 병함하는 기능 추가
# 모델코드로 검색안될시 모델

In [88]:
spec_columns = ["maker","group","code","pl_size",
                "pl_display_inch","pl_weight","pl_battery",
                "pl_front_camera","pl_back_camera"] 

def join_models(df):
    for i in df.index:
        if df.loc[i, "size"] == "":
            for j in df.index:
                if df.loc[j,"group"] == df.loc[i,"group"] and df.loc[j,"size"] != "":
                    df.loc[i,"size"] = df.loc[j,"size"]
                    df.loc[i,"display_inch"] = df.loc[j,"display_inch"]
                    df.loc[i,"weight"] = df.loc[j,"weight"]
                    df.loc[i,"battery"] = df.loc[j,"battery"]
                    df.loc[i,"front_camera"] = df.loc[j,"front_camera"]
                    df.loc[i,"back_camera"] = df.loc[j,"back_camera"]
                    break



In [89]:
join_models(crawl_data)